# feature_selection

El objetivo de este notebook es, una vez generadas las features en `feature_generation` sera elegir cuales son las que mas informacion aportan al modelo, porque hemos generado mas de 300 features, generando una dimensionalidad altisima en nuestro dataset final y seguramente, introduciendo ruido. Eligiendo las mejores k features podremos evitar esto y mejorar resultados.

<hr>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
import nbimporter

import pre_processing
import feature_generation

seed = 7

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb


<hr>

# Mejor alternativa para cada categoria
En `feature_generation` generamos muchas features para cada categoria, con distintos metodos. Por ejemplo, en el caso de `tipodepropiedad`, probamos distintos metodos de encoding. Ahora intentaremos ver, mediante un proceso de seleccion rapida, cual es el mejor metodo para cada caso.

Para probar esto de forma factible (porque si quisieramos probar todas las combinaciones esto requeriria muchisimo timepo) vamos a suponer que nuestras categorias de features son independientes (cosa que sabemos que no es verdad) y analizaremos el caso de cada categoria por separado, para luego quedarnos con la mejor alternativa.

El regresor que utilizaremos para obtener la metrica sera **LightGBM** por su grandiosa velocidad y buenos resultados.

<hr>

### Definimos funciones a utilizar:

In [3]:
def predict(df, columnas, base):
    add = df[columnas]
    train = pd.merge(base, add, how='left', on='id')
    X = train.drop((['precio']), axis=1).values
    Y = train['precio'].values
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)
    d_train = lgb.Dataset(X_train, label=Y_train)
    d_valid = lgb.Dataset(X_val, label=Y_val)
    watchlist = [d_valid]
    reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)
    Y_pred = reg.predict(X_val)
    return mean_absolute_error(Y_val,Y_pred)

In [4]:
def mejor_mae_con_lgb(categoria):
    mejor_mae = 99999999999
    for i in range(len(features[categoria].keys()) - 1):
        mae = predict(df, features[categoria][i], base)
        print(f"Con i={i}, MAE= {mae}")
        if mae<mejor_mae:
            mejor_mae = mae
            mejor = i
    mae_todas = predict(df, features[categoria]['todas'], base)

    resultados_lgb[categoria] = ((mejor, mejor_mae), mae_todas)

In [5]:
def imprimir_resultados(dicc):
    for categoria in features.keys():
        mejor,todas = resultados_lgb[categoria]
        print(f"{categoria} | MEJOR MAE: {mejor[1]} (alternativa: {mejor[0]}) | MAE GLOBAL: {todas}")

<hr>

### Definimos el modelo:

In [6]:
import lightgbm as lgb
# parametros para lograr buena rapidez
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 10, 
    'learning_rate': 0.2,
    'verbose': 0, 
    'early_stopping_round': 20}
n_estimators=10000

### Ejecucion de las pruebas:
Partiremos de un dataset con la informacion basica, al que luego le agregaremos las distintas features.

In [7]:
df,_ = pre_processing.load_featured_datasets()

In [8]:
features = feature_generation.get_features()

In [9]:
base = df[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
           'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
           'centroscomercialescercanos', 'precio']].copy()

In [10]:
# Generamos una nueva alternativa para cada categoria que tenga TODAS las features de dicha categoria:

for feature in features:
    todas = []
    for opcion in features[feature]:
        valores = features[feature][opcion]
        for valor in valores:
            if (valor not in todas):
                todas.append(valor)
    features[feature]['todas'] = todas

In [11]:
resultados_lgb = {}

In [12]:
for categoria in features.keys():
    mejor_mae_con_lgb(categoria)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 1.43139e+06
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l1: 1.29479e+06
[3]	valid_0's l1: 1.19101e+06
[4]	valid_0's l1: 1.10763e+06
[5]	valid_0's l1: 1.04758e+06
[6]	valid_0's l1: 996594
[7]	valid_0's l1: 959312
[8]	valid_0's l1: 926423
[9]	valid_0's l1: 902958
[10]	valid_0's l1: 883026
[11]	valid_0's l1: 865128
[12]	valid_0's l1: 849204
[13]	valid_0's l1: 837011
[14]	valid_0's l1: 824628
[15]	valid_0's l1: 812661
[16]	valid_0's l1: 804177
[17]	valid_0's l1: 794535
[18]	valid_0's l1: 786771
[19]	valid_0's l1: 780731
[20]	valid_0's l1: 776587
[21]	valid_0's l1: 768282
[22]	valid_0's l1: 765080
[23]	valid_0's l1: 760549
[24]	valid_0's l1: 757405
[25]	valid_0's l1: 753889
[26]	valid_0's l1: 751018
[27]	valid_0's l1: 748624
[28]	valid_0's l1: 745009
[29]	valid_0's l1: 742502
[30]	valid_0's l1: 740555
[31]	valid_0's l1: 737292
[32]	valid_0's l1: 735755
[33]	valid_0's l1: 733363
[34]	valid_0's l1: 729852
[35]	valid_0's l1: 728349
[36]	valid_0'

[318]	valid_0's l1: 660806
[319]	valid_0's l1: 660626
[320]	valid_0's l1: 660377
[321]	valid_0's l1: 660379
[322]	valid_0's l1: 660383
[323]	valid_0's l1: 660282
[324]	valid_0's l1: 660253
[325]	valid_0's l1: 660204
[326]	valid_0's l1: 660081
[327]	valid_0's l1: 660045
[328]	valid_0's l1: 660054
[329]	valid_0's l1: 660048
[330]	valid_0's l1: 659952
[331]	valid_0's l1: 659972
[332]	valid_0's l1: 659790
[333]	valid_0's l1: 659711
[334]	valid_0's l1: 659602
[335]	valid_0's l1: 659589
[336]	valid_0's l1: 659511
[337]	valid_0's l1: 659442
[338]	valid_0's l1: 659432
[339]	valid_0's l1: 659371
[340]	valid_0's l1: 659394
[341]	valid_0's l1: 659359
[342]	valid_0's l1: 659258
[343]	valid_0's l1: 659270
[344]	valid_0's l1: 659219
[345]	valid_0's l1: 659130
[346]	valid_0's l1: 658934
[347]	valid_0's l1: 658855
[348]	valid_0's l1: 658859
[349]	valid_0's l1: 658799
[350]	valid_0's l1: 658712
[351]	valid_0's l1: 658685
[352]	valid_0's l1: 658672
[353]	valid_0's l1: 658710
[354]	valid_0's l1: 658689
[

[627]	valid_0's l1: 649740
[628]	valid_0's l1: 649728
[629]	valid_0's l1: 649660
[630]	valid_0's l1: 649653
[631]	valid_0's l1: 649652
[632]	valid_0's l1: 649631
[633]	valid_0's l1: 649569
[634]	valid_0's l1: 649596
[635]	valid_0's l1: 649588
[636]	valid_0's l1: 649562
[637]	valid_0's l1: 649552
[638]	valid_0's l1: 649523
[639]	valid_0's l1: 649549
[640]	valid_0's l1: 649517
[641]	valid_0's l1: 649499
[642]	valid_0's l1: 649443
[643]	valid_0's l1: 649394
[644]	valid_0's l1: 649385
[645]	valid_0's l1: 649304
[646]	valid_0's l1: 649310
[647]	valid_0's l1: 649360
[648]	valid_0's l1: 649414
[649]	valid_0's l1: 649465
[650]	valid_0's l1: 649368
[651]	valid_0's l1: 649382
[652]	valid_0's l1: 649240
[653]	valid_0's l1: 649184
[654]	valid_0's l1: 649122
[655]	valid_0's l1: 649159
[656]	valid_0's l1: 649172
[657]	valid_0's l1: 649136
[658]	valid_0's l1: 649109
[659]	valid_0's l1: 649111
[660]	valid_0's l1: 649037
[661]	valid_0's l1: 649030
[662]	valid_0's l1: 648989
[663]	valid_0's l1: 649002
[

[932]	valid_0's l1: 644847
[933]	valid_0's l1: 644842
[934]	valid_0's l1: 644855
[935]	valid_0's l1: 644863
[936]	valid_0's l1: 644892
[937]	valid_0's l1: 644854
Early stopping, best iteration is:
[917]	valid_0's l1: 644836
Con i=0, MAE= 644836.0032836228


...

[625]	valid_0's l1: 577682
[626]	valid_0's l1: 577640
[627]	valid_0's l1: 577614
[628]	valid_0's l1: 577593
[629]	valid_0's l1: 577592
[630]	valid_0's l1: 577577
[631]	valid_0's l1: 577573
[632]	valid_0's l1: 577557
[633]	valid_0's l1: 577579
[634]	valid_0's l1: 577527
[635]	valid_0's l1: 577485
[636]	valid_0's l1: 577500
[637]	valid_0's l1: 577474
[638]	valid_0's l1: 577479
[639]	valid_0's l1: 577502
[640]	valid_0's l1: 577555
[641]	valid_0's l1: 577548
[642]	valid_0's l1: 577463
[643]	valid_0's l1: 577492
[644]	valid_0's l1: 577464
[645]	valid_0's l1: 577441
[646]	valid_0's l1: 577441
[647]	valid_0's l1: 577378
[648]	valid_0's l1: 577300
[649]	valid_0's l1: 577289
[650]	valid_0's l1: 577301
[651]	valid_0's l1: 577283
[652]	valid_0's l1: 577254
[653]	valid_0's l1: 577227
[654]	valid_0's l1: 577205
[655]	valid_0's l1: 577197
[656]	valid_0's l1: 577180
[657]	valid_0's l1: 577143
[658]	valid_0's l1: 577153
[659]	valid_0's l1: 577117
[660]	valid_0's l1: 577114
[661]	valid_0's l1: 577105
[

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 1.38114e+06
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l1: 1.20985e+06
[3]	valid_0's l1: 1.08093e+06
[4]	valid_0's l1: 984304
[5]	valid_0's l1: 911779
[6]	valid_0's l1: 856760
[7]	valid_0's l1: 817026
[8]	valid_0's l1: 784141
[9]	valid_0's l1: 760115
[10]	valid_0's l1: 740456
[11]	valid_0's l1: 724941
[12]	valid_0's l1: 712145
[13]	valid_0's l1: 702425
[14]	valid_0's l1: 693094
[15]	valid_0's l1: 686231
[16]	valid_0's l1: 680812
[17]	valid_0's l1: 677000
[18]	valid_0's l1: 673573
[19]	valid_0's l1: 669413
[20]	valid_0's l1: 665296
[21]	valid_0's l1: 662952
[22]	valid_0's l1: 660457
[23]	valid_0's l1: 658228
[24]	valid_0's l1: 655771
[25]	valid_0's l1: 653469
[26]	valid_0's l1: 651767
[27]	valid_0's l1: 649216
[28]	valid_0's l1: 646804
[29]	valid_0's l1: 645574
[30]	valid_0's l1: 643620
[31]	valid_0's l1: 642313
[32]	valid_0's l1: 640808
[33]	valid_0's l1: 639697
[34]	valid_0's l1: 638398
[35]	valid_0's l1: 637531
[36]	valid_0's l1: 6368

[314]	valid_0's l1: 587083
[315]	valid_0's l1: 587098
[316]	valid_0's l1: 587126
[317]	valid_0's l1: 587024
[318]	valid_0's l1: 586938
[319]	valid_0's l1: 586941
[320]	valid_0's l1: 586805
[321]	valid_0's l1: 586727
[322]	valid_0's l1: 586686
[323]	valid_0's l1: 586648
[324]	valid_0's l1: 586658
[325]	valid_0's l1: 586627
[326]	valid_0's l1: 586623
[327]	valid_0's l1: 586591
[328]	valid_0's l1: 586640
[329]	valid_0's l1: 586606
[330]	valid_0's l1: 586576
[331]	valid_0's l1: 586577
[332]	valid_0's l1: 586546
[333]	valid_0's l1: 586565
[334]	valid_0's l1: 586497
[335]	valid_0's l1: 586454
[336]	valid_0's l1: 586282
[337]	valid_0's l1: 586234
[338]	valid_0's l1: 586225
[339]	valid_0's l1: 586139
[340]	valid_0's l1: 586099
[341]	valid_0's l1: 585997
[342]	valid_0's l1: 586010
[343]	valid_0's l1: 585989
[344]	valid_0's l1: 585933
[345]	valid_0's l1: 585879
[346]	valid_0's l1: 585800
[347]	valid_0's l1: 585826
[348]	valid_0's l1: 585811
[349]	valid_0's l1: 585739
[350]	valid_0's l1: 585670
[

[629]	valid_0's l1: 577592
[630]	valid_0's l1: 577577
[631]	valid_0's l1: 577573
[632]	valid_0's l1: 577557
[633]	valid_0's l1: 577579
[634]	valid_0's l1: 577527
[635]	valid_0's l1: 577485
[636]	valid_0's l1: 577500
[637]	valid_0's l1: 577474
[638]	valid_0's l1: 577479
[639]	valid_0's l1: 577502
[640]	valid_0's l1: 577555
[641]	valid_0's l1: 577548
[642]	valid_0's l1: 577463
[643]	valid_0's l1: 577492
[644]	valid_0's l1: 577464
[645]	valid_0's l1: 577441
[646]	valid_0's l1: 577441
[647]	valid_0's l1: 577378
[648]	valid_0's l1: 577300
[649]	valid_0's l1: 577289
[650]	valid_0's l1: 577301
[651]	valid_0's l1: 577283
[652]	valid_0's l1: 577254
[653]	valid_0's l1: 577227
[654]	valid_0's l1: 577205
[655]	valid_0's l1: 577197
[656]	valid_0's l1: 577180
[657]	valid_0's l1: 577143
[658]	valid_0's l1: 577153
[659]	valid_0's l1: 577117
[660]	valid_0's l1: 577114
[661]	valid_0's l1: 577105
[662]	valid_0's l1: 577045
[663]	valid_0's l1: 577006
[664]	valid_0's l1: 576994
[665]	valid_0's l1: 576952
[

In [13]:
imprimir_resultados(resultados_lgb)

metros | MEJOR MAE: 640471.9240878306 (alternativa: 1) | MAE GLOBAL: 642119.5412679116
tipodepropiedad | MEJOR MAE: 613117.7133614124 (alternativa: 0) | MAE GLOBAL: 613077.5416794564
provincia | MEJOR MAE: 621839.6424660649 (alternativa: 6) | MAE GLOBAL: 625999.6212025254
ciudad | MEJOR MAE: 610870.6231245665 (alternativa: 2) | MAE GLOBAL: 603870.213767825
fecha | MEJOR MAE: 606334.5691895577 (alternativa: 18) | MAE GLOBAL: 619201.1179593179
descripcion | MEJOR MAE: 637024.5076360593 (alternativa: 0) | MAE GLOBAL: 637024.5076360593
metricas | MEJOR MAE: 590298.0418400187 (alternativa: 2) | MAE GLOBAL: 590298.0418400187
habitaciones | MEJOR MAE: 639429.8556802106 (alternativa: 0) | MAE GLOBAL: 639158.6985959416
antiguedad | MEJOR MAE: 639610.9253228987 (alternativa: 2) | MAE GLOBAL: 645161.1451780535
extras | MEJOR MAE: 609799.4505664627 (alternativa: 2) | MAE GLOBAL: 615707.0979388222
volcanes | MEJOR MAE: 642964.9943022365 (alternativa: 0) | MAE GLOBAL: 642964.9943022365
idzona | MEJO

In [42]:
# En vistas de estos resultados, generamos una funcion para quedarnos con estas categorias.

In [18]:
def get_best_features_per_category():
    return resultados_lgb

# Seleccion de K Mejores Features

In [5]:
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest

In [6]:
train,test = load_featured_datasets()

In [7]:
train.fillna(train.mean(),inplace=True)

In [8]:
X = train.drop('precio', axis=1).values
Y = train['precio'].values

In [9]:
X.shape

(240000, 136)

In [10]:
Selector = SelectKBest(f_regression, k=50)

In [11]:
Selector.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equa

SelectKBest(k=50, score_func=<function f_regression at 0x7f29add1a7b8>)

In [12]:
def k_features_selector(k_features,train_df):
    train_na_filled_df = train_df.fillna(train_df.mean())
    X = train_na_filled_df.drop('precio', axis=1).values
    Y = train_na_filled_df['precio'].values
    Selector = SelectKBest(f_regression, k=k_features)
    Selector.fit(X,Y)
    return Selector